# QIIME 2 enables comprehensive end-to-end analysis of diverse microbiome data and comparative studies with publicly available data

this is a QIIME 2 Artifact API notebook which replicated the QIIME 2 CLI analyses

**environment:** qiime2-2019.10

## How to use this notebook:

1. Activate the `qiime2-2019.10` conda environment.
    ```
    conda activate qiime2-2019.10
    ```

2. Make sure that `jupyter serverextensions` are enabled.  


    Close this notebook and jupyter session, and run:  
    `jupyter serverextension enable --py qiime2 --sys-prefix`  
      
3. Install additional dependencies:
    ```
    conda install songbird -c conda-forge
    conda install -c conda-forge redbiom
    conda install -c bioconda bowtie2
    pip install https://github.com/knights-lab/SHOGUN/archive/master.zip
    pip install https://github.com/qiime2/q2-shogun/archive/master.zip
    conda install cytoolz
    qiime dev refresh-cache
    ```  

4. Restart and run the notebook

# import QIIME 2 plugins and other dependencies

In [1]:
import qiime2
import warnings
warnings.filterwarnings('ignore')

# all plugins that are being used throughout this notebook are imported here
from qiime2.plugins import composition, \
                           deblur, \
                           demux, \
                           diversity, \
                           feature_classifier, \
                           feature_table, \
                           fragment_insertion, \
                           longitudinal, \
                           metadata, \
                           quality_filter, \
                           shogun, \
                           songbird, \
                           taxa

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/drz/Conda/envs/qiime2-2019.10/lib/pyt

## Acquire data from ECAM study 

define the working directory

In [2]:
workdir='./'

In [3]:
!mkdir $workdir/qiime2-ecam-tutorial-api
!cd $workdir/qiime2-ecam-tutorial-api

In [4]:
# NOTE: the file is 1.04GB in size
!wget -O $workdir/81253.zip "https://qiita.ucsd.edu/public_artifact_download/?artifact_id=81253"

--2020-01-13 16:20:30--  https://qiita.ucsd.edu/public_artifact_download/?artifact_id=81253
Resolving qiita.ucsd.edu (qiita.ucsd.edu)... 169.228.46.38
Connecting to qiita.ucsd.edu (qiita.ucsd.edu)|169.228.46.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1116152564 (1.0G) [application/zip]
Saving to: ‘.//81253.zip’

.//81253.zip        100%[===================>]   1.04G  2.89MB/s    in 6m 33s  

2020-01-13 16:27:03 (2.71 MB/s) - ‘.//81253.zip’ saved [1116152564/1116152564]



In [5]:
!unzip $workdir/81253.zip

Archive:  .//81253.zip
 extracting: per_sample_FASTQ/81253/10249.C018.01SS.r.fastq.gz   bad CRC 408b78ca  (should be 82882250)
 extracting: per_sample_FASTQ/81253/10249.C030.01SS.r.fastq.gz   bad CRC 2746113c  (should be 58903356)
 extracting: per_sample_FASTQ/81253/10249.C031.01SS.r.fastq.gz   bad CRC 32768cb1  (should be 46630065)
 extracting: per_sample_FASTQ/81253/10249.C022.01SS.r.fastq.gz   bad CRC c9d8e76c  (should be 86435436)
 extracting: per_sample_FASTQ/81253/10249.C010.01SS.r.fastq.gz   bad CRC 56f95c51  (should be 59182673)
 extracting: per_sample_FASTQ/81253/10249.C014.01SS.r.fastq.gz   bad CRC 549966a6  (should be 19339430)
 extracting: per_sample_FASTQ/81253/10249.C043.01SS.r.fastq.gz   bad CRC 7da85b6d  (should be 08185453)
 extracting: per_sample_FASTQ/81253/10249.C020.01SS.r.fastq.gz   bad CRC 045eb75a  (should be 73316186)
 extracting: per_sample_FASTQ/81253/10249.C035.01SS.r.fastq.gz   bad CRC 4dff8d5c  (should be 08593500)
 extracting: per_sample_FASTQ/81253/10249

 extracting: per_sample_FASTQ/81253/10249.C032.15SS.fastq.gz   bad CRC 3ce7b59f  (should be 21818271)
 extracting: per_sample_FASTQ/81253/10249.M055.01SS.fastq.gz   bad CRC 84748e24  (should be 22231076)
 extracting: per_sample_FASTQ/81253/10249.M043.01SS.fastq.gz   bad CRC 2c4f52b3  (should be 43396019)
 extracting: per_sample_FASTQ/81253/10249.C016.09SS.fastq.gz   bad CRC 0abe65e5  (should be 80250085)
 extracting: per_sample_FASTQ/81253/10249.C044.01SS.fastq.gz   bad CRC d1cd7637  (should be 19903287)
 extracting: per_sample_FASTQ/81253/10249.C007.23SD.fastq.gz   bad CRC 73c8d981  (should be 42542721)
 extracting: per_sample_FASTQ/81253/10249.C010.20SD.fastq.gz   bad CRC 09912103  (should be 60506115)
 extracting: per_sample_FASTQ/81253/10249.C049.09SS.fastq.gz   bad CRC 804ee66f  (should be 52654447)
 extracting: per_sample_FASTQ/81253/10249.M044.01SS.fastq.gz   bad CRC 46ceef24  (should be 87966756)
 extracting: per_sample_FASTQ/81253/10249.C016.01SS.fastq.gz   bad CRC ac09ef79  (

 extracting: per_sample_FASTQ/81253/10249.C046.07SS.fastq.gz   bad CRC a5b95e82  (should be 80388994)
 extracting: per_sample_FASTQ/81253/10249.C037.01SS.fastq.gz   bad CRC fd345391  (should be 48064913)
 extracting: per_sample_FASTQ/81253/10249.C001.01SS.fastq.gz   bad CRC 51be6d84  (should be 71434372)
 extracting: per_sample_FASTQ/81253/10249.C001.34SD.fastq.gz   bad CRC 4691b47c  (should be 83954044)
 extracting: per_sample_FASTQ/81253/10249.M022.01SS.fastq.gz   bad CRC 0519cb82  (should be 85576578)
 extracting: per_sample_FASTQ/81253/10249.C035.01SS.fastq.gz   bad CRC 6f57a5bb  (should be 68015035)
 extracting: per_sample_FASTQ/81253/10249.C034.14SS.fastq.gz   bad CRC 62d2673d  (should be 57956157)
 extracting: per_sample_FASTQ/81253/10249.M024.01SS.fastq.gz   bad CRC 1c50bc17  (should be 75053079)
 extracting: per_sample_FASTQ/81253/10249.C025.08SS.fastq.gz   bad CRC b4b3fce0  (should be 31694560)
 extracting: per_sample_FASTQ/81253/10249.C043.01SS.fastq.gz   bad CRC 7561fe2e  (

 extracting: per_sample_FASTQ/81253/10249.M017.02R.fastq.gz   bad CRC 7229cfe3  (should be 15342819)
 extracting: per_sample_FASTQ/81253/10249.M046.02V.fastq.gz   bad CRC dcc78032  (should be 04062002)
 extracting: per_sample_FASTQ/81253/10249.M034.01R.fastq.gz   bad CRC 0c70c9d9  (should be 08718297)
 extracting: per_sample_FASTQ/81253/10249.M018.03V.fastq.gz   bad CRC e06f4a1c  (should be 65389852)
 extracting: per_sample_FASTQ/81253/10249.M027.01R.fastq.gz   bad CRC 8c0b551c  (should be 49552924)
 extracting: per_sample_FASTQ/81253/10249.M027.03V.fastq.gz   bad CRC 99e5a6ec  (should be 81964524)
 extracting: per_sample_FASTQ/81253/10249.M037.03R.fastq.gz   bad CRC dec271bb  (should be 37285051)
 extracting: per_sample_FASTQ/81253/10249.M007.03R.fastq.gz   bad CRC ece8edba  (should be 74688186)
 extracting: per_sample_FASTQ/81253/10249.M027.03R.fastq.gz   bad CRC 375a07fc  (should be 28647164)
 extracting: per_sample_FASTQ/81253/10249.M046.03V.fastq.gz   bad CRC 90ca2ccb  (should be 

 extracting: per_sample_FASTQ/81253/10249.M036.03V.fastq.gz   bad CRC b095978f  (should be 62593679)
 extracting: per_sample_FASTQ/81253/10249.M033.02R.fastq.gz   bad CRC f6e39d39  (should be 42112057)
 extracting: per_sample_FASTQ/81253/10249.M051.01R.fastq.gz   bad CRC 2eb2e715  (should be 83476501)
 extracting: per_sample_FASTQ/81253/10249.M018.01R.fastq.gz   bad CRC 1bb9352e  (should be 65122606)
 extracting: per_sample_FASTQ/81253/10249.M025.02R.fastq.gz   bad CRC bbfa9af6  (should be 53763062)
 extracting: per_sample_FASTQ/81253/10249.M019.01V.fastq.gz   bad CRC 8c031782  (should be 49012866)
 extracting: per_sample_FASTQ/81253/10249.M015.01R.fastq.gz   bad CRC be59db03  (should be 93559811)
 extracting: per_sample_FASTQ/81253/10249.M043.01R.fastq.gz   bad CRC 0986a258  (should be 59818328)
 extracting: per_sample_FASTQ/81253/10249.M050.01R.fastq.gz   bad CRC 7817e683  (should be 14832259)
 extracting: per_sample_FASTQ/81253/10249.M042.01V.fastq.gz   bad CRC 6d869133  (should be 

In [6]:
!mv $workdir/mapping_files/81253_mapping_file.txt $workdir/metadata.tsv

## Import DNA sequence data into QIIME 2 & create a visual summary

### 1. Create the manifest file with the required column headers

In [7]:
!echo "sample-id\tabsolute-filepath" > manifest.tsv

### 2. Use a loop function to insert the sample names into the sample-id column and add the full paths to the sequence files in the absolute-filepath column

In [8]:
!for f in `ls per_sample_FASTQ/81253/*.gz`; \
do n=`basename $f`; echo "12802.${n%.fastq.gz}\t$PWD/$f"; done >> manifest.tsv

### 3. Use the manifest file to import the sequences into QIIME 2

In [9]:
manifest_single_end = qiime2.Artifact.import_data('SampleData[SequencesWithQuality]',
                                                  view_type='SingleEndFastqManifestPhred33V2',
                                                  view=workdir+'manifest.tsv')

### 4. Create a summary of the demultiplexed artifact

In [10]:
demux_summary = demux.visualizers.summarize(manifest_single_end)

### 5. Visualize feature table

In [11]:
demux_summary.visualization

<visualization: Visualization uuid: 3b338066-30c5-41db-b670-83d876c76d82>

## Import metadata as an object

In [12]:
metadata_ecam = qiime2.Metadata.load(workdir+'/metadata.tsv')

## Sequence quality control and feature table construction

### 1. Apply intial quality filtering 

In [13]:
demux_q_score = quality_filter.methods.q_score(manifest_single_end)

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/q2_quality_filter/_filter.py:88: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  phred_offset = yaml.load(metadata_view)['phred-offset']


### 2. Apply Deblur workflow

In [14]:
# this step is time-consuming
deblur_sequences = deblur.methods.denoise_16S(manifest_single_end,
                                              trim_length=150,
                                              sample_stats=True,
                                              jobs_to_start=1)

### 3. Create a visualization summary of deblur statistics

In [15]:
deblur_viz = deblur.visualizers.visualize_stats(deblur_sequences.stats)
deblur_viz.visualization

<visualization: Visualization uuid: 81deb071-6cda-4127-9886-70e0c6223098>

### 4. Visualize representative sequences

In [16]:
deblur_seq_viz = feature_table.visualizers.tabulate_seqs(deblur_sequences.representative_sequences)
deblur_seq_viz.visualization

<visualization: Visualization uuid: e78747b2-b086-4837-8e5e-76ea37d19ca1>

### 5. Visualize feature table

In [17]:
feature_table_viz = feature_table.visualizers.summarize(deblur_sequences.table,
                                                        metadata_ecam)
feature_table_viz.visualization

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


<visualization: Visualization uuid: e644993d-6d22-4418-abec-b3a1e7de976e>

## Generate a phylogenetic tree

### 1. Download a backbone tree

In [18]:
!wget \
  -O $workdir/sepp-refs-gg-13-8.qza \
  "https://data.qiime2.org/2019.10/common/sepp-refs-gg-13-8.qza"

--2020-01-13 19:55:03--  https://data.qiime2.org/2019.10/common/sepp-refs-gg-13-8.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.10/common/sepp-refs-gg-13-8.qza [following]
--2020-01-13 19:55:03--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.10/common/sepp-refs-gg-13-8.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.144.8
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.144.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50161069 (48M) [binary/octet-stream]
Saving to: ‘.//sepp-refs-gg-13-8.qza’

.//sepp-refs-gg-13- 100%[===================>]  47.84M  3.18MB/s    in 13s     

2020-01-13 19:55:16 (3.72 MB/s) - ‘.//sepp-refs-gg-13-8.qza’ saved [50161069/50161069]



In [19]:
sepp_reference_db = qiime2.Artifact.load(workdir+'sepp-refs-gg-13-8.qza')

### 2. Create an insertion tree

In [20]:
sepp_tree = fragment_insertion.methods.sepp(representative_sequences=deblur_sequences.representative_sequences,
                                            reference_database=sepp_reference_db,
                                            threads=4)

### 3. Filter feature table

In [21]:
filtered_deblur_sequences = fragment_insertion.methods.filter_features(deblur_sequences.table,
                                                                       sepp_tree.tree)

## Taxonomic classification

### 1. Download and import required files

In [25]:
!wget -O $workdir'human-stool.qza' \
https://github.com/BenKaehler/readytowear/raw/master/data/gg_13_8/515f-806r/human-stool.qza

--2020-01-14 09:23:17--  https://github.com/BenKaehler/readytowear/raw/master/data/gg_13_8/515f-806r/human-stool.qza
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/BenKaehler/readytowear/master/data/gg_13_8/515f-806r/human-stool.qza [following]
--2020-01-14 09:23:17--  https://raw.githubusercontent.com/BenKaehler/readytowear/master/data/gg_13_8/515f-806r/human-stool.qza
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.196.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.196.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209813 (205K) [application/octet-stream]
Saving to: ‘./human-stool.qza’

./human-stool.qza   100%[===================>] 204.90K  --.-KB/s    in 0.1s    

2020-01-14 09:23:18 (1.34 MB/s) - ‘./human-stool.qza’ saved [

In [28]:
human_stool = qiime2.Artifact.load(workdir+'human-stool.qza')

In [29]:
!wget -O $workdir'ref-seqs-v4.qza' \
https://github.com/BenKaehler/readytowear/raw/master/data/gg_13_8/515f-806r/ref-seqs-v4.qza

--2020-01-14 09:23:39--  https://github.com/BenKaehler/readytowear/raw/master/data/gg_13_8/515f-806r/ref-seqs-v4.qza
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/BenKaehler/readytowear/master/data/gg_13_8/515f-806r/ref-seqs-v4.qza [following]
--2020-01-14 09:23:40--  https://raw.githubusercontent.com/BenKaehler/readytowear/master/data/gg_13_8/515f-806r/ref-seqs-v4.qza
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.196.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.196.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9319426 (8.9M) [application/octet-stream]
Saving to: ‘./ref-seqs-v4.qza’

./ref-seqs-v4.qza   100%[===================>]   8.89M  3.19MB/s    in 2.8s    

2020-01-14 09:23:44 (3.19 MB/s) - ‘./ref-seqs-v4.qza’ saved 

In [31]:
ref_seqs_v4 = qiime2.Artifact.load(workdir+'ref-seqs-v4.qza')

In [30]:
!wget -O $workdir'ref-tax.qza' \
https://github.com/BenKaehler/readytowear/raw/master/data/gg_13_8/515f-806r/ref-tax.qza

--2020-01-14 09:23:44--  https://github.com/BenKaehler/readytowear/raw/master/data/gg_13_8/515f-806r/ref-tax.qza
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/BenKaehler/readytowear/master/data/gg_13_8/515f-806r/ref-tax.qza [following]
--2020-01-14 09:23:45--  https://raw.githubusercontent.com/BenKaehler/readytowear/master/data/gg_13_8/515f-806r/ref-tax.qza
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.196.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.196.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2604632 (2.5M) [application/octet-stream]
Saving to: ‘./ref-tax.qza’

./ref-tax.qza       100%[===================>]   2.48M  1.18MB/s    in 2.1s    

2020-01-14 09:23:48 (1.18 MB/s) - ‘./ref-tax.qza’ saved [2604632/2604632]



In [32]:
ref_tax = qiime2.Artifact.load(workdir+'ref-tax.qza')

### 2. Train a classifier

In [33]:
human_stool_v4_classifier = feature_classifier.methods.fit_classifier_naive_bayes(ref_seqs_v4,
                                                                                  ref_tax,
                                                                                  human_stool)

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/q2_feature_classifier/classifier.py:102: UserWarning: The TaxonomicClassifier artifact that results from this method was trained using scikit-learn version 0.21.2. It cannot be used with other versions of scikit-learn. (While the classifier may complete successfully, the results will be unreliable.)
  warnings.warn(warning, UserWarning)


### 3. Assign taxonomy

In [34]:
taxonomy = feature_classifier.methods.classify_sklearn(deblur_sequences.representative_sequences,
                                                       human_stool_v4_classifier.classifier)

### 4. Visualize taxonomies

In [35]:
taxonomy_viz = metadata.visualizers.tabulate(taxonomy.classification.view(qiime2.Metadata))
taxonomy_viz.visualization

<visualization: Visualization uuid: 42f28c42-ad4e-4020-8c0e-d371f99adfc2>

## Filter ECAM data to contain children samples only

### 1. Filter feature table

In [36]:
child_only = feature_table.methods.filter_samples(deblur_sequences.table,
                                                  metadata=metadata_ecam,
                                                  where="[mom_or_child]='C'")

### 2. Visualize new feature table

In [37]:
child_only_viz = feature_table.visualizers.summarize(child_only.filtered_table,
                                                     metadata_ecam)
child_only_viz.visualization

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


<visualization: Visualization uuid: 914c6673-5b6c-4459-af1d-67932b559662>

## Alpha rarefaction plots

In [38]:
alpha_rarefaction = diversity.visualizers.alpha_rarefaction(child_only.filtered_table,
                                                            phylogeny=sepp_tree.tree,
                                                            max_depth=10000,
                                                            metadata=metadata_ecam)

## Basic data exploration and diversity analyses

### 0. Filter feature table to include only one sample per subject per month

In [39]:
child_only_norep = feature_table.methods.filter_samples(child_only.filtered_table,
                                                           metadata=metadata_ecam,
                                                           where="[month_replicate]='no'")

In [40]:
child_only_norep_viz = feature_table.visualizers.summarize(child_only_norep.filtered_table,
                                                           metadata_ecam)
child_only_norep_viz.visualization

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


<visualization: Visualization uuid: bc696cf7-1a6f-44db-a91a-728d285cc77e>

### 1. Generate taxonomic barplot

In [41]:
child_taxa = taxa.visualizers.barplot(child_only_norep.filtered_table,
                                      taxonomy.classification,
                                      metadata_ecam)
child_taxa.visualization

<visualization: Visualization uuid: 8ca0b5a4-7750-47bd-a1f8-381c17559b7a>

### 2. Compute alpha and beta diversity

In [42]:
child_only_norep_core_metrics = diversity.pipelines.core_metrics_phylogenetic(child_only_norep.filtered_table,
                                                                              phylogeny=sepp_tree.tree,
                                                                              sampling_depth=3400,
                                                                              metadata=metadata_ecam,
                                                                              n_jobs=4)

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1575: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.010779779321162607 and the largest is 3.5825325637893943.
  RuntimeWarning
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magn

## Perform statistical tests on diversity and generate interactive visualization

### 1. Statistical test on alpha diversity

#### A. Across all time points

In [43]:
shannon_child_only_norep_viz = \
 diversity.visualizers.alpha_group_significance(child_only_norep_core_metrics.shannon_vector,
                                                metadata_ecam)
shannon_child_only_norep_viz.visualization

<visualization: Visualization uuid: fc5928b9-1769-4027-bb9d-8f0cb16b73cc>

#### B. At last time point (month 24)

In [44]:
child_only_norep_C24 = feature_table.methods.filter_samples(child_only_norep.filtered_table,
                                                            metadata=metadata_ecam,
                                                            where="[month]='24'")

In [45]:
child_only_norep_C24_core_metrics = diversity.pipelines.core_metrics_phylogenetic(child_only_norep_C24.filtered_table,
                                                                                  phylogeny=sepp_tree.tree,
                                                                                  sampling_depth=3400,
                                                                                  metadata=metadata_ecam,
                                                                                  n_jobs=1)

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1575: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.02511398635318979 and the largest is 0.8601491353086329.
  RuntimeWarning


In [46]:
shannon_child_only_norep_C24_viz = \
 diversity.visualizers.alpha_group_significance(child_only_norep_C24_core_metrics.shannon_vector,
                                                metadata_ecam)
shannon_child_only_norep_C24_viz.visualization

<visualization: Visualization uuid: cead885b-4245-4797-b153-b1da82b3f350>

### 2. Statistical test on beta diversity

In [47]:
uw_unifrac_delivery_child_only_norep_C24_viz = \
 diversity.visualizers.beta_group_significance(child_only_norep_C24_core_metrics.unweighted_unifrac_distance_matrix,
                                               metadata=metadata_ecam.get_column('delivery'),
                                               pairwise=True)

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/seaborn/categorical.py:410: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_xlim(-.5, len(self.plot_data) - .5, auto=None)


## Longitudinal data analysis

### 1. Linear mixed effects models

In [48]:
child_only_core_metrics = diversity.pipelines.core_metrics_phylogenetic(child_only.filtered_table,
                                                                        phylogeny=sepp_tree.tree,
                                                                        sampling_depth=3400,
                                                                        metadata=metadata_ecam,
                                                                        n_jobs=1)

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1575: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.02229010918297503 and the largest is 5.209201564660603.
  RuntimeWarning
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnit

In [49]:
metadata_ecam_w_shannon = metadata_ecam.merge(child_only_core_metrics.shannon_vector.view(qiime2.Metadata))

In [50]:
lme_shannon_child_only_viz = \
 longitudinal.visualizers.linear_mixed_effects(metadata=metadata_ecam_w_shannon,
                                               metric='shannon',
                                               random_effects='day_of_life',
                                               group_columns='delivery,diet',
                                               state_column='day_of_life',
                                               individual_id_column='host_subject_id')
lme_shannon_child_only_viz.visualization

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2115: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/statsmodels/base/model.p

<visualization: Visualization uuid: 8b0ce6a6-38c5-4b7b-be5b-ea291883e562>

### 2. Volatility visualization

In [51]:
volatility_shannon_child_only_viz = \
 longitudinal.visualizers.volatility(metadata_ecam_w_shannon,
                                     default_metric='shannon',
                                     default_group_column='delivery',
                                     state_column='day_of_life',
                                     individual_id_column='host_subject_id')
volatility_shannon_child_only_viz.visualization

<visualization: Visualization uuid: e2ebda78-3f54-4336-97d2-e29152ccf884>

## Differential abundance testing

### Option 1: ANCOM

In [52]:
# Create a new feature table that contains only samples from children at 6 months
child_only_norep_C6 = feature_table.methods.filter_samples(child_only_norep.filtered_table,
                                                           metadata=metadata_ecam,
                                                           where="[month]='6'")

In [53]:
# filter out low abundant features
filtered_child_only_norep_C6 = feature_table.methods.filter_features(child_only_norep_C6.filtered_table,
                                                                     min_samples=5,
                                                                     min_frequency=20)

In [54]:
# add a pseudocount
composition_table_C6 = composition.methods.add_pseudocount(filtered_child_only_norep_C6.filtered_table)

In [55]:
# run ANCOM
ancom_C6_delivery = composition.visualizers.ancom(composition_table_C6.composition_table,
                                                  metadata_ecam.get_column('delivery'))

### Option 2: songbird

In [56]:
# make a folder to store songbird results
!mkdir $workdir/songbird-results

In [57]:
# run songbird
songbird_norep_C6 = songbird.methods.multinomial(child_only_norep_C6.filtered_table,
                                                 metadata_ecam,
                                                 formula="delivery+abx_exposure+diet+sex",
                                                 epochs=10000,
                                                 differential_prior=0.5)


Instructions for updating:
Use `tf.random.categorical` instead.

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.




/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/biom/table.py:4049: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  for r in self.matrix_data.tocsr()]
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/biom/table.py:4052: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return constructor(mat, index=index, columns=columns)
/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/pandas/core/frame.py:3471: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






  0%|          | 0/60000 [00:00<?, ?it/s]

100%|██████████| 60000/60000 [01:13<00:00, 820.74it/s]


In [58]:
# examine estimated coefficients
songbird_norep_C6.differentials.export_data(workdir+'songbird-results/differentials6monthControlled')

## Meta-analysis through Qiita database using redbiom

NOTE: there is no redbiom Python API, so the commands below are a copy from the CLI notebook

In [59]:
# check the name of contexts and number of samples and features indexed
!redbiom summarize contexts

ContextName	SamplesWithData	FeaturesWithData	Description
Pick_closed-reference_OTUs-Greengenes-Illumina-16S-V34-5c6506	72	2221	Pick closed-reference OTUs (reference-seq: |databases|gg|13_8|rep_set|97_otus.fasta) | Split libraries FASTQ
Deblur-Illumina-16S-V3-V4-150nt-780653	60	869	Deblur (Reference phylogeny for SEPP: Greengenes_13.8, BIOM: reference-hit.biom) | Trimming (length: 150)
Pick_closed-reference_OTUs-Greengenes-IonTorrent-16S-V3-100nt-a243a1	32	1089	Pick closed-reference OTUs (reference-seq: |databases|gg|13_8|rep_set|97_otus.fasta) | Trimming (length: 100)
Pick_closed-reference_OTUs-SILVA-Illumina-16S-V3-54d83f	138	1014	Pick closed-reference OTUs (reference-seq: |projects|qiita_data|reference|silva_119_Silva_119_rep_set97.fna) | Split libraries FASTQ
Pick_closed-reference_OTUs-Greengenes-Illumina-16S-V4-150nt-bd7d4d	137343	71900	Pick closed-reference OTUs (reference-seq: |databases|gg|13_8|rep_set|97_otus.fasta) | Trimming (length: 150)
Pick_closed-reference_OTUs-Greengenes

In [60]:
# identify samples where interested sequence was observed
!redbiom search features --context Deblur-Illumina-16S-V4-150nt-780653 \
TACGTAGGGTGCAAGCGTTATCCGGAATTATTGGGCGTAAAGGGCTCGTAGGCGGTTCGTCGCGTCCGGTGTGAAAGTCCATCGCTTAACGGTGGATCTGCGCCGGGTACGGGCGGGCTGGAGTGCGGTAGGGGAGACTGGAATTCCCGG > observed_samples.txt

In [61]:
# search against only EMP samples
!redbiom summarize samples \
  --category empo_3 \
  --from observed_samples.txt

Animal distal gut	7124
Animal surface	331
Surface (non-saline)	204
Sterile water blank	102
Animal secretion	91
animal distal gut	68
Animal corpus	58
Water (non-saline)	15
Plant corpus	13
Animal proximal gut	12
Aerosol (non-saline)	9
Soil (non-saline)	6
Water (saline)	6
Single strain	6
Sediment (saline)	2
not provided	2
Surface (saline)	1

Total samples	8050


In [62]:
# search against infant samples
!redbiom select samples-from-metadata \
  --context Deblur-Illumina-16S-V4-150nt-780653 \
  --from observed_samples.txt "where (host_age < 3 or age < 3) and qiita_study_id != 10249" > infant_samples.txt

In [63]:
# summarize the metadata of infant samples
!redbiom search metadata \
  --categories birth

!redbiom summarize metadata birth_method birth_mode

!redbiom summarize samples \
     --category birth_mode \
     --from infant_samples.txt

live_births
birth_complications
weight_at_birth
birth_weight_units
birth_length_units
child_1_birth_weight
child_3_preterm_birth
birth_wt
date_of_birth
child_3_birth_weight_unit
birth_wt_sd
child_3_birth_length_unit
place_of_birth
child_1_birth_length
baby_birth_date
child_3_birth_weight
birth_ga_d_units
birth_length
country_of_birth
child_2_birth_weight
birth_ga_w_units
child_3_birth_length
child_2_birth_length_unit
birth_head_cir_units
birth_control
birth
birth_mode
child_2_birth_weight_unit
child_1_birth_weight_unit
birth_weight
birth_season
birth_method
antibiotics_at_birth
birth_location
birth_ga_d
birth_days
child_2_preterm_birth
birth_head_cir
birth_ga_w
still_births
child_1_birth_length_unit
antibiotics_after_birth
birth_year
child_1_preterm_birth
year_of_birth
birth_date
birth_route_2cat
mouse_birth
type_birth_location
child_2_birth_length
birth_method	72
birth_mode	2176
Vaginal	38
Cesarea	16
Vag	3
CSseed	1

Total samples	58


In [64]:
# check sample balance in modes of delivery
!redbiom summarize metadata-category \
  --counter \
  --category birth_mode

Category value	count
Cesarea	47
Vaginal	135
CSseed	335
Vag	689
CS	970


In [65]:
# summarize samples over study id category
!redbiom summarize samples \
  --category qiita_study_id \
  --from infant_samples.txt

10581	54
10918	30
11076	19
1064	15
11358	10
11947	10
2010	4
10512	3
11284	1

Total samples	146


## _Supprot Protocols:_ Exporting QIIME 2 data

A sample export of the SEPP insertion tree

In [66]:
sepp_tree.tree.export_data('extracted-insertion-tree')

## _Support protocols:_ Analysis of shotgun metagenomic data

### 1. Download sample data

In [67]:
!for i in query refseqs taxonomy bt2-database; \
 do wget https://github.com/qiime2/q2-shogun/raw/master/q2_shogun/tests/data/$i.qza; done

--2020-01-14 09:55:32--  https://github.com/qiime2/q2-shogun/raw/master/q2_shogun/tests/data/query.qza
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/qiime2/q2-shogun/master/q2_shogun/tests/data/query.qza [following]
--2020-01-14 09:55:32--  https://raw.githubusercontent.com/qiime2/q2-shogun/master/q2_shogun/tests/data/query.qza
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.196.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.196.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12700 (12K) [application/octet-stream]
Saving to: ‘query.qza’

query.qza           100%[===================>]  12.40K  --.-KB/s    in 0.007s  

2020-01-14 09:55:33 (1.82 MB/s) - ‘query.qza’ saved [12700/12700]

--2020-01-14 09:55:33--  https://github.com/qi

In [68]:
shogun_query = qiime2.Artifact.load(workdir + '/query.qza')

In [69]:
shogun_refseqs = qiime2.Artifact.load(workdir + '/refseqs.qza')

In [70]:
shogun_taxonomy = qiime2.Artifact.load(workdir + '/taxonomy.qza')

In [71]:
bowtie2_db = qiime2.Artifact.load(workdir + '/bt2-database.qza')

### 2. Run shotgun metagenomics pipeline

In [72]:
taxa_table = shogun.methods.nobunaga(query=shogun_query,
                                     reference_reads=shogun_refseqs,
                                     reference_taxonomy=shogun_taxonomy,
                                     database=bowtie2_db)

/home/drz/Conda/envs/qiime2-2019.10/lib/python3.6/site-packages/q2_shogun/_shogun.py:38: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  reftaxa.to_csv(os.path.join(tmpdir, 'taxa.tsv'), sep='\t')


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: shogun align -i /tmp/qiime2-archive-dm3bzaf7/dbea837c-7ad7-4706-8c1f-1155a8c7ee3a/data/dna-sequences.fasta -d /tmp/tmpxt3cvklt -o /tmp/tmpxt3cvklt -a bowtie2 -x 0.8 -t 1 -p 0.98

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: shogun assign_taxonomy -i /tmp/tmpxt3cvklt/alignment.bowtie2.sam -d /tmp/tmpxt3cvklt -o /tmp/tmpxt3cvklt/taxatable.tsv -a bowtie2

